# KNOWLEDGE

The [knowledge](https://github.com/aimacode/aima-python/blob/master/knowledge.py) module covers **Chapter 19: Knowledge in Learning** from Stuart Russel's and Peter Norvig's book *Artificial Intelligence: A Modern Approach*.

Execute the cell below to get started.

In [50]:
from knowledge import *

from notebook import pseudocode, psource

## CONTENTS

* Overview
* Current-Best Learning
* Version-Space Learning

## OVERVIEW

Like the [learning module](https://github.com/aimacode/aima-python/blob/master/learning.ipynb), this chapter focuses on methods for generating a model/hypothesis for a domain. Unlike though the learning chapter, here we use prior knowledge to help us learn from new experiences and find a proper hypothesis.

### First-Order Logic

Usually knowledge in this field is represented as **first-order logic**, a type of logic that uses variables and quantifiers in logical sentences. Hypotheses are represented by logical sentences with variables, while examples are logical sentences with set values instead of variables. The goal is to assign a value to a special first-order logic predicate, called **goal predicate**, for new examples given a hypothesis. We learn this hypothesis by infering knowledge from some given examples.

### Representation

In this module, we use dictionaries to represent examples, with keys the attribute names and values the corresponding example values. Examples also have an extra boolean field, 'GOAL', for the goal predicate. A hypothesis is represented as a list of dictionaries. Each dictionary in that list represents a disjunction. Inside these dictionaries/disjunctions we have conjunctions.

For example, say we want to predict if an animal (cat or dog) will take an umbrella given whether or not it rains or the animal wears a coat. The goal value is 'take an umbrella' and is denoted by the key 'GOAL'. An example:

`{'Species': 'Cat', 'Coat': 'Yes', 'Rain': 'Yes', 'GOAL': True}`

A hypothesis can be the following:

`[{'Species': 'Cat'}]`

which means an animal will take an umbrella if and only if it is a cat.

### Consistency

We say that an example `e` is **consistent** with an hypothesis `h` if the assignment from the hypothesis for `e` is the same as `e['GOAL']`. If the above example and hypothesis are `e` and `h` respectively, then `e` is consistent with `h` since `e['Species'] == 'Cat'`. For `e = {'Species': 'Dog', 'Coat': 'Yes', 'Rain': 'Yes', 'GOAL': True}`, the example is no longer consistent with `h`, since the value assigned to `e` is *False* while `e['GOAL']` is *True*.

## CURRENT-BEST LEARNING

### Overview

In **Current-Best Learning**, we start with a hypothesis and we refine it as we iterate through the examples. For each example, there are three possible outcomes. The example is consistent with the hypothesis, the example is a **false positive** (real value is false but got predicted as true) and **false negative** (real value is true but got predicted as false). Depending on the outcome we refine the hypothesis accordingly:

* Consistent: We do not change the hypothesis and we move on to the next example.

* False Positive: We **specialize** the hypothesis, which means we add a conjunction.

* False Negative: We **generalize** the hypothesis, either by removing a conjunction or a disjunction, or by adding a disjunction.

When specializing and generalizing, we should take care to not create inconsistencies with previous examples. To avoid that caveat, backtracking is needed. Thankfully, there is not just one specialization or generalization, so we have a lot to choose from. We will go through all the specialization/generalizations and we will refine our hypothesis as the first specialization/generalization consistent with all the examples seen up to that point.

### Pseudocode

In [51]:
pseudocode('Current-Best-Learning')

### AIMA3e
__function__ Current-Best-Learning(_examples_, _h_) __returns__ a hypothesis or fail  
&emsp;__if__ _examples_ is empty __then__  
&emsp;&emsp;&emsp;__return__ _h_  
&emsp;_e_ &larr; First(_examples_)  
&emsp;__if__ _e_ is consistent with _h_ __then__  
&emsp;&emsp;&emsp;__return__ Current-Best-Learning(Rest(_examples_), _h_)  
&emsp;__else if__ _e_ is a false positive for _h_ __then__  
&emsp;&emsp;&emsp;__for each__ _h'_ __in__ specializations of _h_ consistent with _examples_ seen so far __do__  
&emsp;&emsp;&emsp;&emsp;&emsp;_h''_ &larr; Current-Best-Learning(Rest(_examples_), _h'_)  
&emsp;&emsp;&emsp;&emsp;&emsp;__if__ _h''_ &ne; _fail_ __then return__ _h''_  
&emsp;__else if__ _e_ is a false negative for _h_ __then__  
&emsp;&emsp;&emsp;__for each__ _h'_ __in__ generalizations of _h_ consistent with _examples_ seen so far __do__  
&emsp;&emsp;&emsp;&emsp;&emsp;_h''_ &larr; Current-Best-Learning(Rest(_examples_), _h'_)  
&emsp;&emsp;&emsp;&emsp;&emsp;__if__ _h''_ &ne; _fail_ __then return__ _h''_  
&emsp;__return__ _fail_  

---
__Figure ??__ The current-best-hypothesis learning algorithm. It searches for a consistent hypothesis that fits all the examples and backtracks when no consistent specialization/generalization can be found. To start the algorithm, any hypothesis can be passed in; it will be specialized or generalized as needed.

### Implementation

As mentioned previously, examples are dictionaries (with keys the attribute names) and hypotheses are lists of dictionaries (each dictionary is a disjunction). Also, in the hypothesis, we denote the *NOT* operation with an exclamation mark (!).

We have functions to calculate the list of all specializations/generalizations, to check if an example is consistent/false positive/false negative with a hypothesis. We also have an auxiliary function to add a disjunction (or operation) to a hypothesis, and two other functions to check consistency of all (or just the negative) examples.

You can read the source by running the cell below:

In [52]:
psource(current_best_learning, specializations, generalizations)

You can view the auxiliary functions in the [knowledge module](https://github.com/aimacode/aima-python/blob/master/knowledge.py). A few notes on the functionality of some of the important methods:

* `specializations`: For each disjunction in the hypothesis, it adds a conjunction for values in the examples encountered so far (if the conjunction is consistent with all the examples). It returns a list of hypotheses.

* `generalizations`: It adds to the list of hypotheses in three phases. First it deletes disjunctions, then it deletes conjunctions and finally it adds a disjunction.

* `add_or`: Used by `generalizations` to add an *or operation* (a disjunction) to the hypothesis. Since the last example is the problematic one which wasn't consistent with the hypothesis, it will model the new disjunction to that example. It creates a disjunction for each combination of attributes in the example and returns the new hypotheses consistent with the negative examples encountered so far. We do not need to check the consistency of positive examples, since they are already consistent with at least one other disjunction in the hypotheses' set, so this new disjunction doesn't affect them. In other words, if the value of a positive example is negative under the disjunction, it doesn't matter since we know there exists a disjunction consistent with the example.

Since the algorithm stops searching the specializations/generalizations after the first consistent hypothesis is found, usually you will get different results each time you run the code.

### Examples

We will take a look at two examples. The first is a trivial one, while the second is a bit more complicated (you can also find it in the book).

First we have the "animals taking umbrellas" example. Here we want to find a hypothesis to predict whether or not an animal will take an umbrella. The attributes are `Species`, `Rain` and `Coat`. The possible values are `[Cat, Dog]`, `[Yes, No]` and `[Yes, No]` respectively. Below we give seven examples (with `GOAL` we denote whether an animal will take an umbrella or not):

In [53]:
animals_umbrellas = [
    {'Species': 'Cat', 'Rain': 'Yes', 'Coat': 'No', 'GOAL': True},
    {'Species': 'Cat', 'Rain': 'Yes', 'Coat': 'Yes', 'GOAL': True},
    {'Species': 'Dog', 'Rain': 'Yes', 'Coat': 'Yes', 'GOAL': True},
    {'Species': 'Dog', 'Rain': 'Yes', 'Coat': 'No', 'GOAL': False},
    {'Species': 'Dog', 'Rain': 'No', 'Coat': 'No', 'GOAL': False},
    {'Species': 'Cat', 'Rain': 'No', 'Coat': 'No', 'GOAL': False},
    {'Species': 'Cat', 'Rain': 'No', 'Coat': 'Yes', 'GOAL': True}
]

Let our initial hypothesis be `[{'Species': 'Cat'}]`. That means every cat will be taking an umbrella. We can see that this is not true, but it doesn't matter since we will refine the hypothesis using the Current-Best algorithm. First, let's see how that initial hypothesis fares to have a point of reference.

In [54]:
initial_h = [{'Species': 'Cat'}]

for e in animals_umbrellas:
    print(guess_value(e, initial_h))

True
True
False
False
False
True
True


We got 5/7 correct. Not terribly bad, but we can do better. Let's run the algorithm and see how that performs.

In [55]:
h = current_best_learning(animals_umbrellas, initial_h)

for e in animals_umbrellas:
    print(guess_value(e, h))

True
True
True
False
False
False
True


We got everything right! Let's print our hypothesis:

In [56]:
print(h)

[{'Species': 'Cat', 'Rain': '!No'}, {'Rain': 'Yes', 'Coat': '!No'}, {'Rain': 'No', 'Coat': 'Yes'}]


If an example meets any of the disjunctions in the list, it will be `True`, otherwise it will be `False`.

Let's move on to a bigger example, the "Restaurant" example from the book. The attributes for each example are the following:

* Alternative option (`Alt`)
* Bar to hang out/wait (`Bar`)
* Day is Friday (`Fri`)
* Is hungry (`Hun`)
* How much does it cost (`Price`, takes values in [$, $$, $$$])
* How many patrons are there (`Pat`, takes values in [None, Some, Full])
* Is raining (`Rain`)
* Has made reservation (`Res`)
* Type of restaurant (`Type`, takes values in [French, Thai, Burger, Italian])
* Estimated waiting time (`Est`, takes values in [0-10, 10-30, 30-60, >60])

We want to predict if someone will wait or not (Goal = WillWait). Below we show twelve examples found in the book.

![restaurant](images/restaurant.png)

With the function `r_example` we will build the dictionary examples:

In [28]:
def r_example(Alt, Bar, Fri, Hun, Pat, Price, Rain, Res, Type, Est, GOAL):
    return {'Alt': Alt, 'Bar': Bar, 'Fri': Fri, 'Hun': Hun, 'Pat': Pat,
            'Price': Price, 'Rain': Rain, 'Res': Res, 'Type': Type, 'Est': Est,
            'GOAL': GOAL}

In code:

In [29]:
restaurant = [
    r_example('Yes', 'No', 'No', 'Yes', 'Some', '$$$', 'No', 'Yes', 'French', '0-10', True),
    r_example('Yes', 'No', 'No', 'Yes', 'Full', '$', 'No', 'No', 'Thai', '30-60', False),
    r_example('No', 'Yes', 'No', 'No', 'Some', '$', 'No', 'No', 'Burger', '0-10', True),
    r_example('Yes', 'No', 'Yes', 'Yes', 'Full', '$', 'Yes', 'No', 'Thai', '10-30', True),
    r_example('Yes', 'No', 'Yes', 'No', 'Full', '$$$', 'No', 'Yes', 'French', '>60', False),
    r_example('No', 'Yes', 'No', 'Yes', 'Some', '$$', 'Yes', 'Yes', 'Italian', '0-10', True),
    r_example('No', 'Yes', 'No', 'No', 'None', '$', 'Yes', 'No', 'Burger', '0-10', False),
    r_example('No', 'No', 'No', 'Yes', 'Some', '$$', 'Yes', 'Yes', 'Thai', '0-10', True),
    r_example('No', 'Yes', 'Yes', 'No', 'Full', '$', 'Yes', 'No', 'Burger', '>60', False),
    r_example('Yes', 'Yes', 'Yes', 'Yes', 'Full', '$$$', 'No', 'Yes', 'Italian', '10-30', False),
    r_example('No', 'No', 'No', 'No', 'None', '$', 'No', 'No', 'Thai', '0-10', False),
    r_example('Yes', 'Yes', 'Yes', 'Yes', 'Full', '$', 'No', 'No', 'Burger', '30-60', True)
]

Say our initial hypothesis is that there should be an alternative option and let's run the algorithm.

In [30]:
initial_h = [{'Alt': 'Yes'}]
h = current_best_learning(restaurant, initial_h)
for e in restaurant:
    print(guess_value(e, h))

True
False
True
True
False
True
False
True
False
False
False
True


The predictions are correct. Let's see the hypothesis that accomplished that:

In [31]:
print(h)

[{'Alt': 'Yes', 'Type': '!Thai', 'Hun': '!No', 'Pat': '!Full'}, {'Alt': 'No', 'Bar': 'Yes', 'Hun': 'No', 'Price': '$', 'Rain': 'No', 'Res': 'No'}, {'Pat': 'Full', 'Price': '$', 'Rain': 'Yes', 'Type': '!Burger'}, {'Price': '$$', 'Type': 'Italian'}, {'Bar': 'No', 'Hun': 'Yes', 'Pat': 'Some', 'Price': '$$', 'Rain': 'Yes', 'Res': 'Yes', 'Type': 'Thai', 'Est': '0-10'}, {'Bar': 'Yes', 'Fri': 'Yes', 'Hun': 'Yes', 'Pat': 'Full', 'Rain': 'No', 'Res': 'No', 'Type': 'Burger'}]


It might be quite complicated, with many disjunctions if we are unlucky, but it will always be correct, as long as a correct hypothesis exists.

## VERSION-SPACE LEARNING

### Overview

**Version-Space Learning** is a general method of learning in logic based domains. We generate the set of all the possible hypotheses in the domain and then we iteratively remove hypotheses inconsistent with the examples. The set of remaining hypotheses is called **version space**. Because hypotheses are being removed until we end up with a set of hypotheses consistent with all the examples, the algorithm is sometimes called **candidate elimination** algorithm.

After we update the set on an example, all the hypotheses in the set are consistent with that example. So, when all the examples have been parsed, all the remaining hypotheses in the set are consistent with all the examples. That means we can pick hypotheses at random and we will always get a valid hypothesis.

### Pseudocode

In [32]:
pseudocode('Version-Space-Learning')

### AIMA3e
__function__ Version-Space-Learning(_examples_) __returns__ a version space  
&emsp;__local variables__: _V_, the version space: the set of all hypotheses  

&emsp;_V_ &larr; the set of all hypotheses  
&emsp;__for each__ example _e_ in _examples_ __do__  
&emsp;&emsp;&emsp;__if__ _V_ is not empty __then__ _V_ &larr; Version-Space-Update(_V_, _e_)  
&emsp;__return__ _V_  

---
__function__ Version-Space-Update(_V_, _e_) __returns__ an updated version space  
&emsp;_V_ &larr; \{_h_ &isin; _V_ : _h_ is consistent with _e_\}  

---
__Figure ??__ The version space learning algorithm. It finds a subset of _V_ that is consistent with all the _examples_.

### Implementation

The set of hypotheses is represented by a list and each hypothesis is represented by a list of dictionaries, each dictionary a disjunction. For each example in the given examples we update the version space with the function `version_space_update`. In the end, we return the version-space.

Before we can start updating the version space, we need to generate it. We do that with the `all_hypotheses` function, which builds a list of all the possible hypotheses (including hypotheses with disjunctions). The function works like this: first it finds the possible values for each attribute (using `values_table`), then it builds all the attribute combinations (and adds them to the hypotheses set) and finally it builds the combinations of all the disjunctions (which in this case are the hypotheses build by the attribute combinations).

You can read the code for all the functions by running the cells below:

In [33]:
psource(version_space_learning, version_space_update)

In [34]:
psource(all_hypotheses, values_table)

In [35]:
psource(build_attr_combinations, build_h_combinations)

### Example

Since the set of all possible hypotheses is enormous and would take a long time to generate, we will come up with another, even smaller domain. We will try and predict whether we will have a party or not given the availability of pizza and soda. Let's do it:

In [36]:
party = [
    {'Pizza': 'Yes', 'Soda': 'No', 'GOAL': True},
    {'Pizza': 'Yes', 'Soda': 'Yes', 'GOAL': True},
    {'Pizza': 'No', 'Soda': 'No', 'GOAL': False}
]

Even though it is obvious that no-pizza no-party, we will run the algorithm and see what other hypotheses are valid.

In [37]:
V = version_space_learning(party)
for e in party:
    guess = False
    for h in V:
        if guess_value(e, h):
            guess = True
            break

    print(guess)

True
True
False


The results are correct for the given examples. Let's take a look at the version space:

In [38]:
print(len(V))

print(V[5])
print(V[10])

print([{'Pizza': 'Yes'}] in V)

959
[{'Pizza': 'Yes'}, {'Soda': 'Yes'}]
[{'Pizza': 'Yes'}, {'Pizza': '!No', 'Soda': 'No'}]
True


There are almost 1000 hypotheses in the set. You can see that even with just two attributes the version space in very large.

Our initial prediction is indeed in the set of hypotheses. Also, the two other random hypotheses we got are consistent with the examples (since they both include the "Pizza is available" disjunction).

## Minimal Consistent Determination

This algorithm is based on a straightforward attempt to find the simplest determination consistent with the observations. A determinaton P > Q says that if any examples match on P, then they must also match on Q. A determination is therefore consistent with a set of examples if every pair that matches on the predicates on the left-hand side also matches on the goal predicate.

### Pseudocode

Lets look at the pseudocode for this algorithm

In [47]:
pseudocode('Minimal-Consistent-Det')

### AIMA3e
__function__ Minimal-Consistent-Det(_E_, _A_) __returns__ a set of attributes  
&emsp;__inputs__: _E_, a set of examples  
&emsp;&emsp;&emsp;&emsp;&emsp;_A_, a set of attributes, of size _n_  

&emsp;__for__ _i_ = 0 __to__ _n_ __do__  
&emsp;&emsp;&emsp;__for each__ subset _A<sub>i</sub>_ of _A_ of size _i_ __do__  
&emsp;&emsp;&emsp;&emsp;&emsp;__if__ Consistent-Det?(_A<sub>i</sub>_, _E_) __then return__ _A<sub>i</sub>_  

---
__function__ Consistent-Det?(_A_, _E_) __returns__ a truth value  
&emsp;__inputs__: _A_, a set of attributes  
&emsp;&emsp;&emsp;&emsp;&emsp;_E_, a set of examples  
&emsp;__local variables__: _H_, a hash table  

&emsp;__for each__ example _e_ __in__ _E_ __do__  
&emsp;&emsp;&emsp;__if__ some example in _H_ has the same values as _e_ for the attributes _A_  
&emsp;&emsp;&emsp;&emsp;but a different classification __then return__ _false_  
&emsp;&emsp;&emsp;store the class of _e_ in_H_, indexed by the values for attributes _A_ of the example _e_  
&emsp;__return__ _true_  

---
__Figure ??__ An algorithm for finding a minimal consistent determination.

You can read the code for the above algorithm by running the cells below:

In [48]:
psource(minimal_consistent_det)

In [49]:
psource(consistent_det)

### Example:

We already know that no-pizza-no-party but we will still check it through the `minimal_consistent_det` algorithm.

In [39]:
print(minimal_consistent_det(party, {'Pizza', 'Soda'}))

{'Pizza'}


We can also check it on some other example. Let's consider the following example :

In [40]:
conductance = [
    {'Sample': 'S1', 'Mass': 12, 'Temp': 26, 'Material': 'Cu', 'Size': 3, 'GOAL': 0.59},
    {'Sample': 'S1', 'Mass': 12, 'Temp': 100, 'Material': 'Cu', 'Size': 3, 'GOAL': 0.57},
    {'Sample': 'S2', 'Mass': 24, 'Temp': 26, 'Material': 'Cu', 'Size': 6, 'GOAL': 0.59},
    {'Sample': 'S3', 'Mass': 12, 'Temp': 26, 'Material': 'Pb', 'Size': 2, 'GOAL': 0.05},
    {'Sample': 'S3', 'Mass': 12, 'Temp': 100, 'Material': 'Pb', 'Size': 2, 'GOAL': 0.04},
    {'Sample': 'S4', 'Mass': 18, 'Temp': 100, 'Material': 'Pb', 'Size': 3, 'GOAL': 0.04},
    {'Sample': 'S4', 'Mass': 18, 'Temp': 100, 'Material': 'Pb', 'Size': 3, 'GOAL': 0.04},
    {'Sample': 'S5', 'Mass': 24, 'Temp': 100, 'Material': 'Pb', 'Size': 4, 'GOAL': 0.04},
    {'Sample': 'S6', 'Mass': 36, 'Temp': 26, 'Material': 'Pb', 'Size': 6, 'GOAL': 0.05},
]



Now, we check the `minimal_consistent_det` algorithm on the above example:

In [41]:
print(minimal_consistent_det(conductance, {'Mass', 'Temp', 'Material', 'Size'}))

{'Temp', 'Material'}


In [43]:
print(minimal_consistent_det(conductance, {'Mass', 'Temp', 'Size'}))


{'Temp', 'Size', 'Mass'}
